In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import shutil
from importlib import reload
import skimage.io
import scipy.io

import matplotlib 
import matplotlib.pyplot as plt 

# local code
import sys
sys.path.append("../source/")

In [2]:
# DATA PATHS
ROOT_DIR = "/nfs/turbo/umms-indikar/shared/projects/live_cell_imaging/PIP_FUCCI_ANALYSIS/WH18/processed/control/"
EXPERIMENT = "control_1"
trackmate_path = f"{ROOT_DIR}{EXPERIMENT}.csv"
tiff_path = f"{ROOT_DIR}{EXPERIMENT}.tif"
OUTPUT_DIR = f"/nfs/turbo/umms-indikar/shared/projects/live_cell_imaging/PIP_FUCCI_ANALYSIS/WH18/processed/control/annotations/{EXPERIMENT}/"

# global vars
# SCALING = 1.4633
SCALING = 0.7310
MINUTES_PER_FRAME = 15

In [3]:
df = pd.read_csv(trackmate_path, skiprows=[1, 2, 3])

print(f"{df.shape=}")
df.head()

df.shape=(19437, 46)


,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,ELLIPSE_X0,ELLIPSE_Y0,ELLIPSE_MAJOR,ELLIPSE_MINOR,ELLIPSE_THETA,ELLIPSE_ASPECTRATIO,AREA,PERIMETER,CIRCULARITY,SOLIDITY
0,ID2048,2048,0.0,0.871527,26.348699,293.307770,0.0,0.0,0,7.813571,...,-0.000851,0.006266,9.217907,6.702768,-0.560894,1.375239,191.800167,55.243380,0.789766,0.949074
1,ID2049,2049,1.0,0.870998,743.289277,578.797845,0.0,0.0,0,9.066239,...,0.033940,0.088550,11.032432,7.528647,-1.472172,1.465394,258.228518,61.117240,0.868734,0.975265
2,ID2050,2050,2.0,0.870230,81.720654,1243.879239,0.0,0.0,0,10.996023,...,0.003171,0.015591,11.406552,10.695609,0.783977,1.066471,379.857892,72.219114,0.915223,0.975962
3,ID2051,2051,3.0,0.870078,810.284137,170.812516,0.0,0.0,0,9.196695,...,0.046963,0.034582,11.691169,7.274885,1.042275,1.607059,265.713402,64.569181,0.800890,0.959459
4,ID2052,2052,4.0,0.869662,828.197972,384.969146,0.0,0.0,0,12.239301,...,-0.008978,-0.000175,13.735521,11.022353,1.185740,1.246151,470.612118,80.917051,0.903219,0.976699


In [4]:
# load the tiff into memory
tiff = skimage.io.imread(tiff_path)
print(f"{tiff.shape=}")

tiff.shape=(52, 1280, 1288, 3)


In [5]:
# define dimensions
t, y, x, c = tiff.shape

In [36]:
def get_counts(points, frame_size, subregion_div):
    """A function to return density estimates for subregions of 
    size (x_subdiv, y_subdiv).
    
    Parameters:
    -----------------------------
        : points (pd.DataFrame): 2 column dataframe with POSITION_X and POSITION_Y
        : frame_size (tuple): the (x, y) size of the total area
        : subregion_div (tuple): the factor by which (x, y) of total area will be divided
        
    Returns:
    -----------------------------
        : density_matrix (2d np.array): a frame_size / subregion_div matrix of densities
        : dipersion (2d np.array): mean distance of points to center of tile
    """
    density_matrix = np.zeros(subregion_div)
    
    subregion_area = int(frame_size[0] / subregion_div[0]) * int(frame_size[1] / subregion_div[1])
    
    x_iter = np.linspace(0, frame_size[0], subregion_div[0]+1, dtype=int)
    y_iter = np.linspace(0, frame_size[1], subregion_div[1]+1, dtype=int)
    
    total_hits = 0
    
    for i, x_coord in enumerate(x_iter[1:]):
        # note that 'i' is the 'x' index - 1
        prev_x_coord = x_iter[i]
        
        for j, y_coord in enumerate(y_iter[1:]):
            # note that 'j' is the 'y' index - 1
            prev_y_coord = y_iter[j]

            mask = (points['POSITION_X'] >= prev_x_coord) & (points['POSITION_X'] < x_coord) & \
                   (points['POSITION_Y'] >= prev_y_coord) & (points['POSITION_Y'] < y_coord)

            hits = points[mask]

            # cell densities
            n_hits = len(hits)
            total_hits += n_hits
            density_matrix[j, i] = n_hits
    return density_matrix

In [43]:
frame_size = (x, y)
subdiv = (10, 10)

xbox = int(x/subdiv[0])
ybox = int(y/subdiv[1])

print(f"{xbox=} {xbox*SCALING=}mm")
print(f"{ybox=} {ybox*SCALING=}mm")
print(f"box area {(xbox*SCALING) * (ybox*SCALING)}mm^2")

Dlist = []

for frame in range(0, df['FRAME'].max()):
    points = df[df['FRAME'] == frame][['POSITION_X', 'POSITION_Y']]
    d = get_counts(points, frame_size, subdiv)
#     print(f"{frame=} {np.sum(d)=}")
    Dlist.append(d)
    
Dmat = np.asarray(Dlist)
print(f"{Dmat.shape=}")

scipy.io.savemat('Dmat.mat', dict(Dmat=Dmat))

xbox=128 xbox*SCALING=93.568mm
ybox=128 ybox*SCALING=93.568mm
box area 8754.970624mm^2
Dmat.shape=(51, 10, 10)
